In [1]:
import pandas as pd
import ast

# Specify the path to your CSV file
csv_file_path = 'scraped_data_reviews_food_full.csv'

# Use pandas read_csv function to read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path, encoding='utf-8')
original_rows = len(df)
print("Number of rows in the original CSV file: %d" % len(df))

# Remove duplicate rows with the same 'href', and adjust the index accordingly
df.drop_duplicates(subset='Review ID', inplace=True)
# Reset the index
df.reset_index(drop=True, inplace=True)

# Now, 'df' is a pandas DataFrame containing the data from the CSV file
# You can perform various operations and analysis on this DataFrame

# Print the first few rows of the DataFrame
print("Number of rows after removing duplicates: %d" % len(df))

print("Number of rows removed: %d" % (original_rows - len(df)))

FileNotFoundError: [Errno 2] No such file or directory: 'scraped_data_reviews_food_full.csv'

In [ ]:
original_rows = len(df)
print("Number of rows in the original CSV file: %d" % len(df))

#limits the number of reviews per place to 5
df['href of Place'] = df.groupby('href of Place').head(5)
#drops all rows with blanks (aka the duplicate rows above 5)
df.dropna(subset=['href of Place'], inplace=True)
df.reset_index(drop=True, inplace=True)

print("Number of rows after removing duplicates: %d" % len(df))
print("Number of rows removed: %d" % (original_rows - len(df)))

print("Categories and their counts:")
print(df['href of Place'].value_counts())

Number of rows in the original CSV file: 63247
Number of rows after removing duplicates: 63128
Number of rows removed: 119
Categories and their counts:
https://www.google.com/maps/place/Sintia%27s+Indonesian+Food/data=!4m7!3m6!1s0x31da1a11943bd381:0x7f162264ca77e695!8m2!3d1.3160429!4d103.8054961!16s%2Fg%2F11j4t63y4b!19sChIJgdM7lBEa2jERleZ3ymQiFn8?authuser=0&hl=en&rclk=1              5
https://www.google.com/maps/place/Guzman+y+Gomez+-+ION+Orchard/data=!4m7!3m6!1s0x31da19d85724d9b9:0xad0cbed308148522!8m2!3d1.3041664!4d103.8317794!16s%2Fg%2F11s94ljrgz!19sChIJudkkV9gZ2jERIoUUCNO-DK0?authuser=0&hl=en&rclk=1            5
https://www.google.com/maps/place/%E4%B8%8D%E9%A5%BF%E4%B8%8D%E5%9D%90/data=!4m7!3m6!1s0x31da3dfe8267b769:0xea52f86208823ba5!8m2!3d1.3933497!4d103.9176661!16s%2Fg%2F11kk7mlqm_!19sChIJabdngv492jERpTuCCGL4Uuo?authuser=0&hl=en&rclk=1    5
https://www.google.com/maps/place/Subway/data=!4m7!3m6!1s0x31da3c8a1d5b365b:0x4d75059b9004bdb4!8m2!3d1.3603912!4d103.989642!16s%2Fg%2F11h9_p

In [ ]:
import pandas as pd
import ast

# Assuming df is your DataFrame and it already exists

# Safely convert the string representation of list of strings into a list of strings
df['Metadata'] = df['Metadata'].apply(ast.literal_eval)

# Create two new columns "Service Rating" and "Service Type"
df['Service Rating'] = None
df['Service Type'] = None

# Function to process and return the correct value based on the input
def process_value(value, key):
    if "$" in value or key == "Price per person":
        value = value.replace("$", "").replace("RM ", "")
        if "+" in value:
            value = value.replace("+", "")
            return float(value)*1.5
        if "–" in value:
            parts = value.split("–")
            return sum(float(part) for part in parts) / len(parts)
        return float(value)
    try:
        return int(value)
    except ValueError:
        return value

def rating_or_type(value):
    # if integer, then it's a rating
    try:
        int(value)
        return "Service Rating"
    except ValueError:
        return "Service Type"

# Iterate over each row and update the DataFrame with new columns for each key-value pair
for index, metadata_list in enumerate(df['Metadata']):
    for item in metadata_list:
        try:
            key, value = item.split(": ", 1)
            if key == "Service":
                key = rating_or_type(value)
            df.at[index, key] = process_value(value, key)
        except ValueError:
            # print(f"Key has no value: {item}")
            pass

# Drop the columns in which there is less than 20% of non-null values
df.dropna(thresh=len(df) * 0.2, axis=1, inplace=True)


# Drop the original 'Metadata' column
df.drop(columns=['Metadata'], inplace=True)

df

,href of Place,Review ID,Relavancy Ranking,Reviewer href,Reviewer Name,Local Guide,Total Reviews,Total Photos,Star Rating,Date,Review,Likes,Review Images href,Service Rating,Service Type,Food,Atmosphere,Meal type,Price per person,Recommended dishes
0,https://www.google.com/maps/place/One+More+Bow...,ChZDSUhNMG9nS0VJQ0FnSURGXy1peFZREAE,1,https://www.google.com/maps/contrib/1157070012...,Yi Ling,True,524,3850,4,2023-12-02 10:01:11.963467,Would say it's a relatively good bowl of dry m...,0,['https://lh5.googleusercontent.com/p/AF1QipNd...,3,None,3.0,4.0,NaN,NaN,NaN
1,https://www.google.com/maps/place/One+More+Bow...,ChdDSUhNMG9nS0VJQ0FnSUQyME5YZnJRRRAB,2,https://www.google.com/maps/contrib/1143455552...,CK C,True,1863,5617,5,2023-02-02 10:01:12.483552,Dry Mee Hoon Kueh was very tasty and generous ...,3,['https://lh5.googleusercontent.com/p/AF1QipPI...,None,None,NaN,NaN,NaN,NaN,NaN
2,https://www.google.com/maps/place/Lian+Kee/dat...,ChdDSUhNMG9nS0VJQ0FnSUNacWFPZnp3RRAB,1,https://www.google.com/maps/contrib/1043278646...,Daphne Seow,True,360,4251,5,2023-12-02 10:01:12.730551,One of my favourite soup noodles places! Love ...,1,['https://lh5.googleusercontent.com/p/AF1QipOD...,5,Dine in,5.0,5.0,Breakfast,5.5,"Fishball Noodle, Fishball Soup, Meatball Soup,..."
3,https://www.google.com/maps/place/One+More+Bow...,ChdDSUhNMG9nS0VJQ0FnSUQtckp2UDd3RRAB,3,https://www.google.com/maps/contrib/1127702228...,William Amedeo Chong,True,45,51,5,2023-02-02 10:01:13.066097,This has to be the no.1 dry mee hoon kueh I ha...,1,['https://lh5.googleusercontent.com/p/AF1QipNW...,5,None,5.0,4.0,NaN,NaN,NaN
4,https://www.google.com/maps/place/One+More+Bow...,ChdDSUhNMG9nS0VJQ0FnSURPbDhqTC1nRRAB,4,https://www.google.com/maps/contrib/1118800275...,Kuan,True,85,59,4,2023-02-02 10:01:13.779937,Mee hoon Kueh texture is good. Q-Q and not sog...,0,['https://lh5.googleusercontent.com/p/AF1QipNG...,None,Dine in,NaN,NaN,Lunch,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63123,https://www.google.com/maps/place/Wok+Express+...,ChdDSUhNMG9nS0VJQ0FnSURSaDllMXJnRRAB,1,https://www.google.com/maps/contrib/1074762720...,Nicholas Tan,True,83,208,4,2023-06-03 00:20:39.441882,The food is pretty nice and average filling fo...,0,['https://lh5.googleusercontent.com/p/AF1QipPo...,4,None,4.0,3.0,NaN,NaN,NaN
63124,https://www.google.com/maps/place/Wok+Express+...,ChZDSUhNMG9nS0VJQ0FnSUN4NE5MT0FnEAE,2,https://www.google.com/maps/contrib/1090930684...,Elaine Lee,False,4,13,1,2023-06-03 00:20:39.943853,Ordered take out on 8 May 2023. After eating h...,2,['https://lh5.googleusercontent.com/p/AF1QipMD...,None,Take out,NaN,NaN,Dinner,5.5,NaN
63125,https://www.google.com/maps/place/Wok+Express+...,ChZDSUhNMG9nS0VJQ0FnSUNaMUx1SFF3EAE,3,https://www.google.com/maps/contrib/1020535585...,Adriana Zul,False,Local,17,4,2023-10-03 00:20:40.525909,1 meat + 4 other toppings costed me $7. ok to ...,1,[],4,Take out,4.0,3.0,Lunch,5.5,"Egg Fried Rice, Sous Vide Egg, Vegetable Toppings"
63126,https://www.google.com/maps/place/Wok+Express+...,ChdDSUhNMG9nS0VJQ0FnSUNla2JPbTRRRRAB,4,https://www.google.com/maps/contrib/1030788709...,Yang,True,852,16757,4,2023-02-03 00:20:40.955822,Wok Hei strong here.\nFried rice is nice.\nMay...,1,['https://lh5.googleusercontent.com/p/AF1QipOX...,None,Take out,NaN,NaN,Lunch,NaN,NaN


In [ ]:
# Process the 'Date' column to keep only the date part
df['Date'] = pd.to_datetime(df['Date']).dt.date

# Display the date part of df
df['Date']

0        2023-12-02
1        2023-02-02
2        2023-12-02
3        2023-02-02
4        2023-02-02
            ...    
63123    2023-06-03
63124    2023-06-03
63125    2023-10-03
63126    2023-02-03
63127    2023-02-03
Name: Date, Length: 63128, dtype: object

In [ ]:
# Write to a new CSV file
df.to_csv('scraped_data_reviews_food_processed.csv', encoding='utf-8-sig',index=False)